In [173]:
# def dice_avg(dice_size) -> float:
#     return (dice_size + 1) / 2.0
# 
# dice_avg(4), dice_avg(6), dice_avg(8), dice_avg(10), dice_avg(12)

In [174]:
import math
import random

ATTACK_ROLL_DICE_SIZE = 20
BASE_ABILITY_SIZE = 10
DEFAULT_TARGET_AC = 15


def roll_dice(dice_size=ATTACK_ROLL_DICE_SIZE) -> int:
    return random.randint(1, dice_size)


[roll_dice() for _ in range(10)]


[6, 5, 3, 7, 2, 9, 16, 7, 11, 14]

In [175]:
# def base_weapon_damage_avg(dice_size, dice_count=1, bonus=0):
#     return (dice_avg(dice_size) + bonus) * dice_count
# 
# base_weapon_damage_avg(10, bonus=1), base_weapon_damage_avg(6, bonus=1)

In [176]:
from dataclasses import dataclass


@dataclass
class Weapon:
    name: str
    dice_size: int
    dice_count: int = 1
    bonus: int = 0

    def damage_roll(self, critical=False) -> int:
        dice_count = self.dice_count * 2 if critical else self.dice_count
        return sum([roll_dice(self.dice_size) for _ in range(dice_count)]) + self.bonus


hand_crossbow_0 = Weapon("Hand Crossbow", dice_size=6)
hand_crossbow_1 = Weapon("Hand Crossbow +1", dice_size=6, bonus=1)
hand_crossbow_2 = Weapon("Hand Crossbow +2", dice_size=6, bonus=2)
heavy_crossbow_0 = Weapon("Heavy Crossbow", dice_size=10)
heavy_crossbow_1 = Weapon("Heavy Crossbow +1", dice_size=10, bonus=1)
heavy_crossbow_2 = Weapon("Heavy Crossbow +2", dice_size=10, bonus=2)
hand_crossbow_0.damage_roll(), hand_crossbow_0.damage_roll(), hand_crossbow_0.damage_roll(), hand_crossbow_0.damage_roll(
    critical=True)

(2, 2, 4, 5)

In [177]:
longbow_0 = Weapon("Longbow", dice_size=8)
longbow_1 = Weapon("Longbow", dice_size=8, bonus=1)
longbow_2 = Weapon("Longbow", dice_size=8, bonus=2)
longbow_3 = Weapon("Longbow", dice_size=8, bonus=3)
longbow_3.damage_roll(), longbow_3.damage_roll(critical=True)

(9, 5)

In [178]:
heavy_crossbow_2.damage_roll(critical=True), heavy_crossbow_2.damage_roll(critical=True), heavy_crossbow_2.damage_roll(
    critical=True),

(7, 14, 10)

In [179]:
proficiency_bonus_on_level = {
    1: 2,
    2: 2,
    3: 2,
    4: 2,
    5: 3,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 4
}

In [180]:
main_ability_on_level_default = {
    1: 17,
    2: 17,
    3: 18,
    4: 18,
    5: 18,
    6: 19,
    7: 19,
    8: 19,
    9: 20,
    10: 20,
    11: 20,
    12: 21
}

In [181]:
import sys
import logging
from pathlib import Path

logger = logging.Logger(">>")
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s %(name)s [%(levelname)s]: %(message)s')
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

path_to_log = Path(r"C:\Users\slafniy\Desktop\bg3modding\DedTunedProject\Data\Projects\CommonUtility\damage_calculator.log")
file_handler = logging.FileHandler(path_to_log)
logger.addHandler(file_handler)

In [182]:
class Character:
    def __init__(self, name: str,
                 level: int,
                 weapon_main: Weapon,
                 weapon_offhand: Weapon = None):
        self.name = name
        self.level = level
        self.weapon_main = weapon_main
        self.weapon_offhand = weapon_offhand
        self.base_proficiency_bonus = proficiency_bonus_on_level[level]
        self.ability_proficiency_bonus = int(math.floor((main_ability_on_level_default[level] - BASE_ABILITY_SIZE) / 2))

    def do_attack(self, weapon: Weapon, target_ac) -> int:
        attack_roll = roll_dice(ATTACK_ROLL_DICE_SIZE)
        if attack_roll == 1:
            logger.info(f"{self.name} -> 0 damage, Critical miss!")
            return 0

        if attack_roll == ATTACK_ROLL_DICE_SIZE:
            res = weapon.damage_roll(critical=True)
            logger.info(f"{self.name} -> {res} damage, Critical hit!")
            return res
        
        attack_roll = attack_roll + self.base_proficiency_bonus + self.ability_proficiency_bonus + weapon.bonus
        if attack_roll< target_ac:
            logger.info(f"{self.name} -> 0 damage, rolled {attack_roll} against {target_ac}")
            return 0
        
        res = weapon.damage_roll()
        logger.info(f"{self.name} -> {res} damage, rolled {attack_roll} against {target_ac}")
        return res

    def main_hand_attack(self, target_ac=DEFAULT_TARGET_AC):
        return self.do_attack(self.weapon_main, target_ac)
    
    def offhand_attack(self, target_ac=DEFAULT_TARGET_AC):
        return self.do_attack(self.weapon_offhand, target_ac)



In [184]:
lvl2_hand_crossbow_shooter = Character("Hand crossbow user", 2, hand_crossbow_1, hand_crossbow_1)
 
[lvl2_hand_crossbow_shooter.main_hand_attack() for _ in range(20)]

2024-11-09 20:34:23,378 >> [INFO]: Hand crossbow user -> 0 damage, rolled 8 against 15
2024-11-09 20:34:23,379 >> [INFO]: Hand crossbow user -> 0 damage, rolled 11 against 15
2024-11-09 20:34:23,379 >> [INFO]: Hand crossbow user -> 5 damage, rolled 23 against 15
2024-11-09 20:34:23,379 >> [INFO]: Hand crossbow user -> 4 damage, rolled 21 against 15
2024-11-09 20:34:23,380 >> [INFO]: Hand crossbow user -> 0 damage, rolled 8 against 15
2024-11-09 20:34:23,380 >> [INFO]: Hand crossbow user -> 0 damage, rolled 13 against 15
2024-11-09 20:34:23,381 >> [INFO]: Hand crossbow user -> 4 damage, rolled 22 against 15
2024-11-09 20:34:23,381 >> [INFO]: Hand crossbow user -> 5 damage, rolled 25 against 15
2024-11-09 20:34:23,381 >> [INFO]: Hand crossbow user -> 3 damage, rolled 20 against 15
2024-11-09 20:34:23,382 >> [INFO]: Hand crossbow user -> 5 damage, rolled 18 against 15
2024-11-09 20:34:23,382 >> [INFO]: Hand crossbow user -> 6 damage, rolled 16 against 15
2024-11-09 20:34:23,382 >> [INFO]:

[0, 0, 5, 4, 0, 0, 4, 5, 3, 5, 6, 8, 2, 7, 7, 4, 6, 3, 0, 2]